In [1]:
import assembly_game
import gymnasium as gym

game = gym.make("Min2Game")

game.reset()

(((1, 2, 0, 0), (2, 1, 0, 0)), {})

Let's first move the value from %rdi to the return value (%rax), as you can see we immediately get reward of 10 because in one training example result is in the correct spot \
we are getting however the penalty of 1 for every timestep.

In [3]:
from assembly_game.environments import Instruction, Operand

In [4]:
game.step((Instruction.MOV, Operand.RDI, Operand.RAX))

([(1, 2, 1, 0), (2, 1, 2, 0)], 9, False, False, {})

Let's now compare the value with the value in %rsi, if $rax happens to be greater then it must be the case that the value %rsi is minimum

In [5]:
game.step((Instruction.CMP, Operand.RSI, Operand.RAX))

([(1, 2, 1, -1), (2, 1, 2, 1)], 8, False, False, {})

After conditional move all testing examples are "solved", as we are getting reward of 20 and penalty of 3 instructions used

In [6]:
game.step((Instruction.CMOVG, Operand.RSI, Operand.RAX))

([(1, 2, 1, -1), (2, 1, 1, 1)], 17, False, False, {})

Lastly we need to run the RET instructions to observe value of terminated=True

In [7]:
game.step((Instruction.RET, None, None))

([(1, 2, 1, -1), (2, 1, 1, 1)], 16, True, False, {})

# Now let's see how to add timelimit to the environment

In [ ]:
game = gym.make("Min2Game", max_episode_steps=5)